In [23]:
import numpy as np
import random
from matplotlib import pyplot as plt
from tkinter import *
from tkinter import ttk
from turtle import *

In [55]:
# paramètres
#taille monde:

n = 10
m = 20
nb_tour = 15
#taux d'infection :

rzero = 2

#durée de vie virus et vaccin :

dureeinfection = 3
dureevaccin = 12

#taux vaccinal :

taux_immun = 0.95

freq_vaccin = 1
freq_depla = 4

In [25]:
#génère un monde de taille n,m

def generate_spatial_world(n,m):
    return np.random.choice([0,None],size=(n,m),p = [0.9,0.1])
worldtest = generate_spatial_world(n,m)
worldtest

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, None, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [26]:
#génère la premiere infection aléatoire d'un monde de taille n,m

def infection_world(world):
    worldbis = np.copy(world)
    while True:
        n = random.randint(0,len(worldbis[:,0])-1)
        m = random.randint(0,len(worldbis[0,:])-1)
        if worldbis[n,m] == 0:
            worldbis[n,m] = 1
            break
    return worldbis

worldtest=infection_world(worldtest)

worldtest

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [1, None, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [27]:
#infecte la cellule x,y, si elle n'est pas vaccinée

def infection_cellule(world,x,y):
    worldbis = np.copy(world)
    if world[x,y] != 2:
        worldbis[x,y] = 1
    return worldbis

worldtest = infection_cellule(worldtest,1,1)
worldtest

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [1, None, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [28]:
def infection_vaccin(world,parallele,x,y,taux_immun):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    if world[x,y] == 2:
        chance = random.random()
        if chance >= taux_immun:
            worldbis[x,y] = 1
            parabis[x,y] = 0
    return worldbis,parabis



In [29]:
# idée pour la gestion des temps d'infection /vaccin , création d'un deuxieme vecteur 2d comportant les durées de vie des états actuels

#création d'un monde parallele:

def creation_parallele(world):
    parallele = np.zeros((len(world[:,0]),len(world[0,:])))
    return parallele

#test
#print(creation_parallele(worldtest))
#print(worldtest)
#paralleletest = creation_parallele(worldtest2)

In [30]:

#création d'un monde parallele:

def desinfection(world,duree_infection,parallele):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    for i in range(len(world[:,0])):
        for j in range(len(world[0,:])):
            if (worldbis[i][j] == 1 and parabis[i][j] != duree_infection):
                parabis[i][j] += 1
            elif(worldbis[i][j] == 1 and parabis[i][j] == duree_infection):
                worldbis[i][j] = 0
                parabis[i][j] = 0
    return worldbis , parabis

#test
#ix = 0
#for ix in range (4):
    #worldtest2,paralleletest = desinfection(worldtest2,3,paralleletest )
    #print(worldtest2,paralleletest)
#print(desinfection(worldtest2,duréeinfection,creation_parallele(worldtest2)))

In [31]:
#génère un tableau en 1d du voisinage de la case nm

def voisins(world,n,m):
    voisins=[]
    for i in range(0,len(world[:,0])):
        for j in range(0,len(world[0,:])):
            if max(abs(i-n),abs(j-m))<=1 and not(i==n and j == m) :
                voisins.append(world[i,j])
    return voisins

In [32]:
#vérifie si il y autre chose que des malades et cases vides dans le voisinage

def verif_voisins(world,n,m):
    for i in voisins(world,n,m):
        if i != None and i!= 1:
            return True
    return False

In [33]:
#fais évoluer le monde en effectuant un tour( pour le moment seul les nouvels infections sont gérées)
#avec la proba
def tour(world,rzero,duree_infection,parallele):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    for i in range (0, len(world[:,0])):
        for j in range ( 0, len(world[0,:])):
            if world[i,j] == 1 and verif_voisins(worldbis,i,j):
                nb = 0 
                while nb < rzero and verif_voisins(worldbis,i,j):
                    x = i + (random.randint(-1,1))
                    y = j + (random.randint(-1,1))
                    if (x >= 0 and x <= len(world[:,0])-1) and (y>= 0 and y<= len(world[0,:])-1):
                        if worldbis[x,y] != None and worldbis[x,y]!= 1:
                            worldbis = infection_cellule(worldbis,x,y)
                            nb += 1
    return desinfection(worldbis,duree_infection,parabis)


In [34]:
#effectue le deplacement d'un membre du monde choisi de manière aléatoire
#tp
def deplacement(world,parallele):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    x1 = random.randint(0,len(world[:,0])-1)
    y1 = random.randint(0,len(world[0,:])-1)
    x2 = random.randint(0,len(world[:,0])-1)
    y2 = random.randint(0,len(world[0,:])-1)  
    while world[x1,y1] == None or (x1 == x2 and y1 == y2):
        x1 = random.randint(0,len(world[:,0])-1)
        y1 = random.randint(0,len(world[0,:])-1) 
    worldbis[x2,y2] = world[x1,y1]
    worldbis[x1,y1] = world[x2,y2]
    parabis[x2,y2] = parallele[x1,y1]
    parabis[x1,y1] = parallele[x2,y2]
    return worldbis,parabis

In [35]:
#effectue x tour d'évolution du monde en ajoutant un déplacement aléatoire tous les freq_depla tours 

def tour_boucle(world,rzero,nbtour,freq_depla,duree_infection,parallele):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    i = 0
    while i < nbtour:
        if i%freq_depla == 0:
            worldbis,parabis = deplacement(worldbis,parabis)
        worldbis,parabis = tour(worldbis,rzero,duree_infection,parabis)
        #print(worldbis,"\n",parabis,"\n","\n")
        i+= 1
    return worldbis

In [36]:
def vaccin(world):
    worldbis = np.copy(world)
    breack = 0
    for i in range (0, len(world[:,0])):
        if breack != 0:
            break
        for j in range ( 0, len(world[0,:])):
            if world[i][j] == 0 :
                breack +=1
                break
    if breack == 0:
        return worldbis
    else:
        x1 = random.randint(0,len(world[:,0])-1)
        y1 = random.randint(0,len(world[0,:])-1) 
        while world[x1,y1] != 0:
            x1 = random.randint(0,len(world[:,0])-1)
            y1 = random.randint(0,len(world[0,:])-1) 
        worldbis[x1,y1] = 2
        return worldbis

vaccin(worldtest)

array([[0, 0, 2, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [1, None, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [37]:
def devaccin(world,parallele,duree_vaccin):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    for i in range(len(worldbis[:,0])):
        for j in range(len(worldbis[0,:])):
            if (worldbis[i][j] == 2 and parabis[i][j] != duree_vaccin):
                parabis[i][j] += 1
            elif(worldbis[i][j] == 2 and parabis[i][j] == duree_vaccin):
                worldbis[i][j] = 0
                parabis[i][j] = 0
    return worldbis , parabis

In [38]:
def tour_vaccin(world,rzero,duree_infection,parallele,duree_vaccin,taux_immun):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    for i in range (0, len(world[:,0])):
        for j in range ( 0, len(world[0,:])):
            if world[i,j] == 1 and verif_voisins(worldbis,i,j):
                nb = 0 
                while nb < rzero and verif_voisins(worldbis,i,j):
                    x = i + (random.randint(-1,1))
                    y = j + (random.randint(-1,1))
                    if (x >= 0 and x <= len(worldbis[:,0])-1) and (y>= 0 and y<= len(worldbis[0,:])-1):
                        if worldbis[x,y] == 0:
                            worldbis = infection_cellule(worldbis,x,y)
                            nb += 1
                        if worldbis[x,y] == 2:
                            worldbis,parabis = infection_vaccin(worldbis,parabis,x,y,taux_immun)
                            nb+=1
    worldbis,parabis = desinfection(worldbis,duree_infection,parabis)
    worldbis,parabis = devaccin(worldbis,parabis,duree_vaccin)
    return worldbis, parabis



In [39]:
def tour_boucle_vaccin(world,rzero,nbtour,freq_depla,duree_infection,parallele,duree_vaccin,freq_vaccin,taux_immun):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    i = 0
    while i < nbtour:
        print(worldbis,"\n",parabis,"\n","\n")
        if i%freq_depla == 0:
            worldbis,parabis = deplacement(worldbis,parabis)
        if i%freq_vaccin == 0:
            worldbis = vaccin(worldbis)
        worldbis,parabis = tour_vaccin(worldbis,rzero,duree_infection,parabis,duree_vaccin,taux_immun)
        i+= 1
    return worldbis

In [60]:
def cycle_entier(n,m,nbtour,freq_depla,rzero,duree_infection,duree_vaccin,freq_vaccin,taux_immun):
    world = generate_spatial_world(n,m)
    world = infection_world(world)
    worldun = np.copy(world)import Tkinter as Tk
import random

class Schelling():
	"""A Schelling model.
	This is a model of a city undergoing dynamic racial segregation."""

	race_colors = ["#F9C", "#000", "#FF0", "#F00", "#533"]

	def __init__(self, width, height, races = 2):
		self.width = width 
		self.height = height 
		self.races = races
		self.race_array = [[0] * height for x in range(width)] 
		self.tk_array = [[None] * height for x in range(width)] 
		self.empty_spaces = []

	def populate(self):
		self.empty_spaces = []
		for x in range(self.width):
			for y in range(self.height):
				if random.randint(0, 9) == 0:
					self.race_array[x][y] = 0
					self.empty_spaces.append((x, y))
				else:
					self.race_array[x][y] = random.randint(1, self.races)

	def use_canvas(self, canvas, w, h):
		"""Fills the canvas with objects"""
		self.canvas = canvas
		self.canvas_w = w
		self.canvas_h = h
		tile_w = self.canvas_w / self.width
		tile_h = self.canvas_h / self.height
		for x in range(self.width):
			for y in range(self.height):
				if self.race_array[x][y] == 0:
					self.tk_array[x][y] = None
				else:
					race = self.race_array[x][y]
					self.tk_array[x][y] = \
						self.canvas.create_rectangle(x * tile_w, y * tile_h,
								   (x+1) * tile_w, (y+1) * tile_h,
								   fill=Schelling.race_colors[race - 1])

	def update(self, n):
		"""Perform N iterations of the is_unhappy check."""
		for i in range(n):
			x = random.randint(0, self.width - 1)
			y = random.randint(0, self.height - 1)
			if self.is_unhappy(x,y):
				self.move_to_empty(x, y)

	def move_to_empty(self, x1, y1):
		"""Moves to an empty cell."""
		new_cell = random.randint(0, len(self.empty_spaces) - 1)
		x2, y2 = self.empty_spaces[new_cell]

		self.race_array[x1][y1], self.race_array[x2][y2] = self.race_array[x2][y2], self.race_array[x1][y1]

		tile_w = self.canvas_w / self.width
		tile_h = self.canvas_h / self.height

		self.canvas.coords(self.tk_array[x1][y1], x2 * tile_w, y2 * tile_h, (x2+1) * tile_w, (y2+1) * tile_h)

		self.tk_array[x1][y1], self.tk_array[x2][y2] = self.tk_array[x2][y2], self.tk_array[x1][y1]

		self.empty_spaces[new_cell] = (x1,y1)

	def is_unhappy(self, x, y):
		"""A square is unhappy if it does not have at least two similar neighbours.
		Empty squares are never unhappy."""
		me = self.race_array[x][y]
		if me == 0:
			return False
		count = 0
		if x > 0 and self.race_array[x-1][y] == me:
			count += 1
		if x < self.width - 1 and self.race_array[x+1][y] == me:
			count += 1
		if y > 0 and self.race_array[x][y-1] == me:
			count += 1
		if y < self.height - 1 and self.race_array[x][y+1] == me:
			count += 1
		return count < 2

class SchellingApp():
	def __init__(self, w = 400, h = 400):
		self.root = Tk.Tk()
		self.w = w
		self.h = h
		self.simulation = Schelling(100, 100, 4)
		self.simulation.populate()
		self.canvas = Tk.Canvas(self.root, width = w, height = h)
		self.canvas.grid(column = 0, row = 0)
		self.simulation.use_canvas(self.canvas, w, h)
		self.canvas.after(8, self.update)
		self.root.mainloop()

	def update(self):
		self.simulation.update(20)
		self.canvas.after(8, self.update)

if __name__ == '__main__':
	app = SchellingApp()
    worldeux = np.copy(world)
    paraun = creation_parallele(world)
    paradeux = creation_parallele(world)
    
    world,parallele = desinfection(world,duree_infection,paraun)
    worldeux,paradeux = desinfection(worldeux,duree_infection,paradeux)
    
    world = tour_boucle(worldun,rzero,nbtour,freq_depla,duree_infection,paraun)
    worldeux = tour_boucle_vaccin(worldeux,rzero,nbtour,freq_depla,duree_infection,paradeux,duree_vaccin,freq_vaccin,taux_immun)
    return world , worldeux
    
cycle_entier(n,m,nb_tour,freq_depla,rzero,dureeinfection,dureevaccin,freq_vaccin,taux_immun)

def print_cycle_entier(n,m,nbtour,freq_depla,rzero,duree_infection,duree_vaccin,freq_vaccin,taux_immun):
    return cycle_entier(n,m,nbtour,freq_depla,rzero,duree_infection,duree_vaccin,freq_vaccin,taux_immun)


[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 None 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 None 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 None 0 0 0 0]
 [0 0 0 0 0 None 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 None 0 0 0 0 None 0 None 0 None 0 0 0 0 0]
 [0 0 0 0 0 0 None 0 0 0 None None 0 0 None 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 None 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 None 0 None 0 None 0 0 0 0 0 0 None]] 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0

In [61]:
#essaie tkinter
def fenetre():
    fenetre = Tk()
    fenetre.geometry('800x800')
    fenetre.title('Sars-CoV2')
    fenetre['bg'] = 'black'
    start = Button(fenetre, text ="Simuler" , command =lambda: print_cycle_entier(n,m,nb_tour,freq_depla,rzero,dureeinfection,dureevaccin,freq_vaccin,taux_immun))
    start.pack()
    end = Button(fenetre ,text = "FERMER" , command = fenetre.destroy)
    end.pack()
    fenetre.mainloop()
fenetre()


[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 None 0 0 0 0 0]
 [0 0 0 0 0 0 None 0 0 0 0 0 0 0 0 None 0 0 0 None]
 [0 0 0 None 0 0 0 0 0 None 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 None None 0 0 1 0]
 [0 0 0 0 0 0 0 None None 0 0 0 0 0 0 0 0 0 0 0]
 [None None 0 None 0 0 0 0 None 0 0 None 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 None 0 0 0 0]
 [0 0 0 0 None 0 0 None 0 0 0 None 0 0 0 0 None 0 0 0]
 [None 0 0 0 0 0 0 0 0 0 0 0 None 0 0 0 0 0 0 0]
 [None 0 0 0 0 0 0 None 0 0 0 0 0 0 None 0 0 0 0 None]] 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [62]:
#test 
	race_colors = ["#F9C", "#000", "#FF0", "#F00"]

	def __init__(self, width, height, races = 2):
		self.width = width 
		self.height = height 
		self.races = races
		self.race_array = [[0] * height for x in range(width)] 
		self.tk_array = [[None] * height for x in range(width)] 
		self.empty_spaces = []
    
    def populate(self):
		self.empty_spaces = []
		for x in range(self.width):
			for y in range(self.height):
				if world[x][y] == None:
					self.race_array[x][y] = 0
					self.empty_spaces.append((x, y))
				else:
					self.race_array[x][y] = random.randint(1, self.races)

	def use_canvas(self, canvas, w, h):
		"""Fills the canvas with objects"""
		self.canvas = canvas
		self.canvas_w = w
		self.canvas_h = h
		tile_w = self.canvas_w / self.width
		tile_h = self.canvas_h / self.height
		for x in range(self.width):
			for y in range(self.height):
				if self.race_array[x][y] == 0:
					self.tk_array[x][y] = None
				else:
					race = self.race_array[x][y]
					self.tk_array[x][y] = \
						self.canvas.create_rectangle(x * tile_w, y * tile_h,
								   (x+1) * tile_w, (y+1) * tile_h,
								   fill=Schelling.race_colors[race - 1])

ModuleNotFoundError: No module named 'Tkinter'